In [28]:

# from sklearn import tree
import pandas as pd
import numpy as np


In [29]:
def normalize(X):
    for i in range(X.shape[1]):
        X[:,i] = (X[:,i] - X[:,i].mean())/X[:,i].std()
    return X

In [30]:
k = 10
p = 8
def get_data():
    """
    Returns the data from the xlsx file
    """
    file_name = 'yeast.csv' 
    df = pd.read_csv(file_name, index_col=0, header=None)
    # print(df.head()) # print the first 5 rows
    return df

md = get_data()
md.dropna(inplace = True)
print(md.shape)
md.replace('?', 0, inplace = True)

print(md)

(1484, 9)
               1     2     3     4    5    6     7     8    9
0                                                            
ADT1_YEAST  0.58  0.61  0.47  0.13  0.5  0.0  0.48  0.22  MIT
ADT2_YEAST  0.43  0.67  0.48  0.27  0.5  0.0  0.53  0.22  MIT
ADT3_YEAST  0.64  0.62  0.49  0.15  0.5  0.0  0.53  0.22  MIT
AAR2_YEAST  0.58  0.44  0.57  0.13  0.5  0.0  0.54  0.22  NUC
AATM_YEAST  0.42  0.44  0.48  0.54  0.5  0.0  0.48  0.22  MIT
...          ...   ...   ...   ...  ...  ...   ...   ...  ...
YUR1_YEAST  0.81  0.62  0.43  0.17  0.5  0.0  0.53  0.22  ME2
ZIP1_YEAST  0.47  0.43  0.61  0.40  0.5  0.0  0.48  0.47  NUC
ZNRP_YEAST  0.67  0.57  0.36  0.19  0.5  0.0  0.56  0.22  ME2
ZUO1_YEAST  0.43  0.40  0.60  0.16  0.5  0.0  0.53  0.39  NUC
G6PD_YEAST  0.65  0.54  0.54  0.13  0.5  0.0  0.53  0.22  CYT

[1484 rows x 9 columns]


In [31]:
X = md.iloc[:, 0:8].values.reshape(-1, 8)


print(X)
print(X.shape)
# add 1 to the first column
# X = np.insert(X, 0, 1, axis=1)
# np.random.shuffle(X)

print('X:', X)
print(X.shape)

X = normalize(X)
print(X)

M = X.shape[0]
Y = md.iloc[:, 8:9].values.reshape(-1, 1)


[[0.58 0.61 0.47 ... 0.   0.48 0.22]
 [0.43 0.67 0.48 ... 0.   0.53 0.22]
 [0.64 0.62 0.49 ... 0.   0.53 0.22]
 ...
 [0.67 0.57 0.36 ... 0.   0.56 0.22]
 [0.43 0.4  0.6  ... 0.   0.53 0.39]
 [0.65 0.54 0.54 ... 0.   0.53 0.22]]
(1484, 8)
X: [[0.58 0.61 0.47 ... 0.   0.48 0.22]
 [0.43 0.67 0.48 ... 0.   0.53 0.22]
 [0.64 0.62 0.49 ... 0.   0.53 0.22]
 ...
 [0.67 0.57 0.36 ... 0.   0.56 0.22]
 [0.43 0.4  0.6  ... 0.   0.53 0.39]
 [0.65 0.54 0.54 ... 0.   0.53 0.22]]
(1484, 8)
[[ 0.58198136  0.88848148 -0.3466451  ... -0.0991314  -0.34417514
  -0.5279193 ]
 [-0.51089067  1.37281104 -0.23122636 ... -0.0991314   0.52121948
  -0.5279193 ]
 [ 1.01913017  0.96920307 -0.11580762 ... -0.0991314   0.52121948
  -0.5279193 ]
 ...
 [ 1.23770457  0.5655951  -1.61625127 ... -0.0991314   1.04045625
  -0.5279193 ]
 [-0.51089067 -0.80667199  1.15379854 ... -0.0991314   0.52121948
   1.06900494]
 [ 1.0919883   0.32343032  0.46128609 ... -0.0991314   0.52121948
  -0.5279193 ]]


In [32]:
def p(i):
    return 1 / (1 + np.exp(-i))

In [33]:
outputs = ['CYT', 'NUC', 'MIT', 'ME3', 'ME2', 'ME1', 'EXC', 'VAC', 'POX', 'ERL']

print('results', Y[0] == outputs[2])
y = np.zeros((len(Y), 10))
for i in range(len(Y)):
        for j in range(10):
            if outputs[j] == Y[i]:
                  y[i][j] = 1
            else:
                  y[i][j] = 0
print(y.shape)
# for i in range(10):
#     print(outputs[i], ': ', y[i])

results [ True]
(1484, 10)


In [34]:
eta = 1
beta = np.zeros((10, 8))

for i in range(10):
    beta[i] = np.random.rand(1, 8)

print(beta)
gradient = np.zeros((10, 8))
trainNo = 1000
loopCount = 1000

n = 0
while (True):
    for i in range(trainNo):   
        tmp = p(X[i] @ beta.T)
        tmp = tmp - y[i]
        for t in range(k):
            gradient[t] = gradient[t] + tmp[t] * X[i]
    gradient = gradient / trainNo
    
    
    beta = beta - eta * gradient
    print('g: ', gradient, 'norm', np.linalg.norm(gradient))

    n = n + 1
    if n == trainNo or np.linalg.norm(gradient) < 0.0097:
        break
    eta = eta * 0.99

print('beta: ', beta)

[[0.94788664 0.61841895 0.83793518 0.44402633 0.3624407  0.58665905
  0.81605926 0.98991826]
 [0.82681898 0.57068878 0.72321941 0.96110591 0.58549527 0.18563401
  0.54893557 0.58994769]
 [0.45569755 0.37056204 0.63623218 0.36356081 0.58540782 0.1001105
  0.10453815 0.0410287 ]
 [0.59771579 0.71657747 0.64283738 0.10532917 0.12892169 0.04097182
  0.5085347  0.43998154]
 [0.38806594 0.02941895 0.50633387 0.21069451 0.54496343 0.3325535
  0.78457245 0.67424054]
 [0.33977535 0.51885861 0.64515002 0.60701303 0.81744568 0.90299342
  0.03354521 0.27770465]
 [0.40811167 0.29636725 0.15629283 0.56112166 0.90118895 0.82988313
  0.45654431 0.67083961]
 [0.12872429 0.40994485 0.51293144 0.47170597 0.22549451 0.64639467
  0.30941594 0.59118565]
 [0.60005009 0.14134458 0.14500553 0.12469455 0.64428721 0.65982204
  0.38049304 0.19156527]
 [0.90385439 0.69687882 0.68415838 0.59765948 0.92730604 0.83123743
  0.83822921 0.51932969]]
g:  [[ 0.22092043  0.2130782  -0.0412806   0.21786891  0.05994725  0.08

In [35]:
# eta = 0.001
# trainNo = 1200
# beta = np.zeros((10, 9))
# for t in range(10):
#     for i in range(trainNo):
#         # take one sample randomly
#         # i = np.random.randint(0, trainNo)
#         print(i)
        
#         # calculate the gradient
#         for k in range(10):
#             for j in range(9):
#                 gradient[k][j] = (p(np.dot(X[i], beta[k])) - y[i][k]) * X[i][j]

#         print('grad', gradient)

#         # update beta
#     for k in range (10):1
#         beta[k] = beta[k] - eta * gradient[k] / trainNo

In [37]:
# one or not classification
y_pred = np.zeros(M - trainNo)
estimates = np.zeros(10)

for i in range(M - trainNo):
    for t in range(10):
        estimates[t] = p(np.dot(X[trainNo + i], beta[t]))
    y_pred[i] = np.argmax(estimates)
    # print(estimates)
    # print(x_pred[i])
# print(y_pred)
# print(type(y_pred[0]))
for i in range(M - trainNo):
    print(outputs[int(y_pred[i])], end=' ')

correct = 0
for i in range(M - trainNo):
    if np.where(y[trainNo + i] == 1) == y_pred[i]:
        correct = correct + 1
print(correct / (M - trainNo), (M - trainNo))


<class 'numpy.float64'>
MIT MIT MIT NUC MIT NUC CYT CYT CYT CYT MIT CYT CYT CYT MIT MIT CYT EXC MIT MIT NUC CYT CYT NUC MIT MIT NUC NUC NUC CYT NUC NUC MIT ME3 NUC NUC EXC CYT NUC MIT CYT ME3 MIT CYT MIT MIT MIT MIT CYT NUC ME2 ME3 CYT CYT CYT CYT CYT ME3 ERL MIT ME3 ME3 MIT NUC ME3 CYT ME3 ME3 ME3 CYT ME3 ME3 CYT NUC NUC CYT EXC ERL ME3 NUC ME3 NUC CYT MIT EXC ME2 ME3 MIT ME3 ME3 MIT NUC NUC CYT MIT MIT NUC NUC NUC CYT NUC ME3 ME3 CYT NUC NUC NUC MIT ME3 CYT NUC ME3 NUC ME3 NUC ME3 ME3 CYT MIT ME3 ME3 MIT NUC ME3 ME3 NUC NUC CYT CYT CYT ME3 CYT MIT NUC ME3 NUC CYT CYT CYT NUC ME3 NUC MIT MIT ME3 POX NUC ME3 ME3 NUC CYT ME3 ME3 CYT NUC NUC NUC CYT CYT ME3 CYT CYT CYT NUC MIT NUC ME3 NUC NUC NUC NUC NUC CYT CYT CYT CYT NUC NUC CYT MIT NUC NUC NUC ME3 ME3 MIT ME3 ME3 EXC NUC ME3 ME3 NUC ME3 NUC ME2 NUC NUC ME3 MIT NUC NUC ME3 NUC ME3 ME3 ME3 EXC EXC EXC CYT CYT CYT NUC ME3 CYT NUC ME3 MIT MIT ME3 CYT NUC NUC CYT ME3 ME3 EXC ME3 ME3 NUC ME3 CYT CYT ME2 ME3 CYT MIT MIT MIT NUC CYT CYT CYT 